In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_chinese import ChatZhipuAI, ZhipuAIEmbeddings
from langchain_chinese import (
    LocalFilesLoader,
    create_qa_chain,
    create_qa_toolkits,
    create_reason_agent,
)
from langchain_community.vectorstores import Chroma

docs = LocalFilesLoader()
chunks = docs.load_and_split()

db = Chroma.from_documents(chunks, ZhipuAIEmbeddings())

chain = create_qa_chain(ChatZhipuAI(), db.as_retriever())

In [6]:
chain.invoke("供应商达标的业绩要求是什么")

AIMessage(content='根据提供的资料，供应商的达标业绩要求并未在文本中直接列出。但是，可以通过以下几点推断出一些关于业绩的要求：\n\n1. 财务能力：供应商应具备健全和稳定的财务状况，以及良好的信用记录。这意味着供应商需要有稳定的财务表现和良好的信用历史。\n\n2. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。这表明供应商需要在合作期间达到最低销售额要求。\n\n3. 流动资金：供应商应具备足够的流动资金来支持合同执行。这意味着供应商需要有足够的资金来保证合同的顺利执行。\n\n总的来说，供应商的达标业绩要求包括良好的财务状况、信用记录以及满足最低销售额要求等。')

In [4]:
async for chunk in chain.astream_events("供应商达标的业绩要求是什么?", version="v1"):
	# print(chunk)
	# print(chunk['event'])
	if(chunk['event'] == "on_chat_model_stream"):
		print(chunk['data']['chunk'].content, end="|", flush=True)
	elif(chunk['event'] == "on_retriever_end"):
		for doc in chunk['data']['output']['documents']:
			print(doc.metadata['source'])


/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/求职简历.docx
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
供应商|达|标的|业绩|要求|包括|以下|几点|：|

1|.| 月|销售额|：|与我|方|合作的|月份|，|经|我|方|销售|的产品|月|总价|不得|低于|人民币|3|万元|。|
2|.| |稳定的|财务|状况|：|供应商|应|具备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。|
3|.| |流动|资金|：|供应商|应|具备|足够的|流动|资金|来|支持|合同|执行|。||

In [3]:
llm_zhipu = ChatZhipuAI(model="glm-4")
tools = create_qa_toolkits(qa_chain=chain)
agent = create_reason_agent(llm_zhipu, tools=tools)

In [4]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [6]:
agent_executor.invoke({"input":"供应商怎样才算合格？"})

{'input': '供应商怎样才算合格？',
 'output': '供应商是否合格取决于其在供应商评价体系各个维度上的表现，以及公司设定的具体合格标准。这些标准可能包括资质齐全、经营和财务能力良好、产品和服务质量高、行业经验和声誉良好、具有社会责任感和可持续性。具体的合格分数线可能需要根据公司的实际情况和需求进行调整。'}

In [5]:
async for chunk in agent_executor.astream_events({"input": "供应商怎样才算合格？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="", flush=True)

/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


### 关键概念:
- 供应商合格标准

### 概念拆解:
- 供应商合格标准 - 
  - 供应的产品质量
  - 供应的时效性
  - 供应商的信誉度
  - 供应商的管理水平

### 反思:
- 需要先了解供应商合格的具体标准，才能判断一个供应商是否合格。
- 上述提到的几个概念拆解可能并不完整，需要根据实际情况进一步查询相关信息。

### 思考:
- 首先需要了解的是供应商合格的具体标准，这是一切判断的基础。
- 在没有更多信息的情况下，我不能对以上任何一个子要素做出有效的查询计划。
- 我需要尝试使用现有工具获取这些信息。

### 推理:
- 由于缺乏具体的信息源，我将尝试使用ask_document工具获取供应商合格标准的相关信息。

### 计划:
- 使用ask_document工具，查询关于供应商合格标准的问题。

### 输出所选择执行的动作/工具/指令:
```json
{"name": "ask_document", "args": {"query": "供应商合格标准是什么？"}}
```### 关键概念:
- 供应商合格标准：已具备合法有效的营业执照、税务登记证、组织机构代码证；财务状况健全稳定；信用记录良好；流动资金充足；产品质量符合国家和行业标准；服务体系高标准；至少五年相关行业经验；良好的业务声誉和客户满意度；环保法规遵守；社会责任积极承担。

### 概念拆解:
- 供应商合格标准
  - 合法有效的证件
  - 财务状况
  - 信用记录
  - 流动资金
  - 产品质量
  - 服务体系
  - 行业经验
  - 业务声誉
  - 客户满意度
  - 环保法规遵守
  - 社会责任承担

### 反思:
- 已经得到了供应商合格的具体标准，这是判断供应商是否合格的基础。
- 所获得的信息似乎涵盖了供应商合格的主要标准，无需进一步查询。

### 思考:
- 现在已经拥有了供应商合格标准的详细信息，下一步是依据这些标准来判断一个特定的供应商是否合格。
- 需要确保所有标准都有明确的判断依据，不可以有任何模糊的地方。
- 在当前阶段，没有新的工具需要调用，因为已经拥有了所有必要的信息。

### 推理:
- 由于拥有了供应商合格的所有标准，可以开始准备一个评估流程来判断特定供应商是否符合这些标准。

### 计划:
- 